In [18]:
#!pip install kfp
import sys
sys.path.append('../../../igxartifacts/lib')
sys.path.append('../../../pipelines/sdk/python')

In [19]:
import kfp
from kfp import dsl

In [20]:
import os
import sys

# Convert notebooks to python

In [21]:
from igz import v3io_cred, xcp_op, mount_v3io

In [22]:
NOTEBOOKS_DIR = os.getenv('NOTEBOOKS_DIR','/User/GitHub/tutorials/demos/location_based_recommendation')
N_GEN = os.getenv('N_GEN','02-nuclio-generator')
WORK_DIR = os.getenv('WORK_DIR','/v3io/bigdata/lbr')
LBR_PATH  = os.getenv('LBR_PATH','/v3io/bigdata/lbr')

N_GEN_CMD = "/lbr/" + N_GEN + ".py"

In [23]:
print(NOTEBOOKS_DIR)

/User/GitHub/tutorials/demos/location_based_recommendation


In [24]:
%env NOTEBOOKS_DIR={NOTEBOOKS_DIR}
%env N_GEN={N_GEN}
%env LGBM_NB={LGBM_NB}

env: NOTEBOOKS_DIR=/User/GitHub/tutorials/demos/location_based_recommendation
env: N_GEN=02-nuclio-generator
env: LGBM_NB={LGBM_NB}


In [9]:
!jupyter nbconvert --output-dir={WORK_DIR} --to python {NOTEBOOKS_DIR}/{N_GEN}.py
#!echo {NOTEBOOKS_DIR}/{PREP_NB}

[NbConvertApp] Converting notebook /User/GitHub/tutorials/demos/location_based_recommendation/02-nuclio-generator.ipynb to python
[NbConvertApp] Writing 4671 bytes to /v3io/bigdata/lbr/02-nuclio-generator.py


In [37]:
!cp {NOTEBOOKS_DIR}/{N_GEN}.ipynb {LBR_PATH}/.

In [38]:
nuclio_deploy = kfp.components.load_component(url='https://raw.githubusercontent.com/kubeflow/pipelines/master/components/nuclio/deploy/component.yaml')
nuclio_invoke = kfp.components.load_component(url='https://raw.githubusercontent.com/kubeflow/pipelines/master/components/nuclio/invoker/component.yaml')

In [39]:
def run_step(NAME,CMD,LBR_PATH):
    """Print env vars."""
    cop = dsl.ContainerOp(
        name=NAME,
        image='python:3.6-jessie',
        command=['ls /lbr',CMD],
    )
    cop.apply(v3io_cred())
    cop.apply(mount_v3io(container='bigdata', sub_path=LBR_PATH, mount_path='/lbr'))
    return cop

def run_python(NAME,CMD):
    """Print env vars."""
    cop = dsl.ContainerOp(
        name=NAME,
        image='python:3.6-jessie',
        command=['python',CMD],
    )
    cop.apply(v3io_cred())
    cop.apply(mount_v3io(container='bigdata', sub_path='/lbr', mount_path='/lbr'))
    return cop

In [40]:
@dsl.pipeline(
    name='SCRATCH pipeline',
    description='Runnign LBR in kubeflow'
)
def my_pipeline(
   txt='good morningf',
   val=7,
):
    #build = run_python('N_GEN',N_GEN_CMD)
    build = nuclio_deploy(url='/lbr/02-nuclio-generator.ipynb', name='generator1', project='myproj',dashboard='http://default-tenant-nuclio-dashboard:8070').apply(mount_v3io(container='bigdata', sub_path='/lbr', mount_path='/lbr'))


    

In [41]:
kfp.compiler.Compiler().compile(my_pipeline, 'scratchpipe.yaml')

In [42]:
EXPERIMENT_NAME = 'Scratch pipeline'

In [43]:
client = kfp.Client(namespace='default-tenant')
arguments = {'txt': 'good evening', 'val': 7}
experiment = client.create_experiment(EXPERIMENT_NAME)
run_name = my_pipeline.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, 'scratchpipe.yaml', arguments)